In [ ]:
import dci_lite.client
from datetime import datetime
client = dci_lite.client.DCIClient()

In [ ]:
openstack_team = client.teams.first(where='name:OpenStack')

In [ ]:
import pandas as pd
from dateutil.relativedelta import relativedelta

datelist = pd.date_range(start='2016/01/01', end=pd.datetime.today() + relativedelta(weeks=1), freq='W').tolist()

month_idx = 0

teams = {}

for job in client.jobs.list(sort="created_at", where='status:success'):
    if not (job.team_id() and job.topic_id()):
        continue
    if openstack_team.id() != job.team.parent_id():
        continue
    if not job.status != 'success':
        continue
    if not job.topic.name().startswith('OSP'):
        continue
    if job.team.external() is False:
        continue
    if job.team.name() not in teams:
        teams[job.team.name()] = [0 for _ in datelist]

    while not (
            datelist[month_idx].year == job.created_at().year and 
            datelist[month_idx].isocalendar()[1] == job.created_at().isocalendar()[1]):
        month_idx += 1
    teams[job.team.name()][month_idx] += 1

In [ ]:
print(teams.keys())

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

team_names = teams.keys()
sorted(team_names)

for team in team_names:
    plt.figure(figsize=(9,9))
    plt.title(team)
    plt.plot_date(datelist[:-1], teams[team][:-1], '-', label=team)
    plt.xlabel('Weeks')
    plt.ylabel('Successful runs')
plt.show()